# Vehículo - Inicio

In [ ]:
import time
from threading import Thread
from .memory import Memory
from .log import get_logger

logger = get_logger(__name__)

class Vehicle:
    def __init__(self, mem=None):
        if not mem:
            mem = Memory()
        self.mem = mem
        self.parts = []
        self.on = True
        self.threads = []

    def add(self, part, inputs=[], outputs=[],
            threaded=False, run_condition=None):
        """
        Método para agregar a la conducción del vehículo.

         Parámetros
         ----------
             inputs : list
                 Nombres de Channel para obtener de la memoria.
             outputs : list
                 Nombres de Channel para guardar en la memoria.
             threaded : boolean
                 Si una parte debe ejecutarse en un thread separado.
             run_condition: boolean
                 Si una parte debe ejecutarse.
  
        """
        p = part
        logger.info('Adding part {}.'.format(p.__class__.__name__))
        entry = dict()
        entry['part'] = p
        entry['inputs'] = inputs
        entry['outputs'] = outputs
        entry['run_condition'] = run_condition

        if threaded:
            t = Thread(target=part.update, args=())
            t.daemon = True
            entry['thread'] = t
        self.parts.append(entry)

    def start(self, rate_hz=10, max_loop_count=None):
        """
        Inicio del circuito de conducción principal del vehículo.
        Este es el thread principal del vehículo. Inicia todos los subprocesos nuevos, luego inicia 
        un bucle infinito y actualiza la memoria.
         
         Parámetros
         ----------
         rate_hz: int
        La frecuencia máxima con la que debe funcionar el bucle. 
         max_loop_count: int
        Número máximo de bucles. Esto se utiliza para probar todas las partes del funcionamiento del vehículo.
         
        """

        try:
            self.on = True

            for entry in self.parts:
                if entry.get('thread'):
                    # iniciar el thread de actualización
                    entry.get('thread').start()

            # espera hasta que las piezas se activan.
            logger.info('Starting vehicle...')
            time.sleep(1)

            loop_count = 0
            while self.on:
                start_time = time.time()
                loop_count += 1

                self.update_parts()

                # detener el bucle si loop_count excede max_loopcount
                if max_loop_count and loop_count > max_loop_count:
                    self.on = False

                sleep_time = 1.0 / rate_hz - (time.time() - start_time)
                if sleep_time > 0.0:
                    time.sleep(sleep_time)

        except KeyboardInterrupt:
            pass
        finally:
            self.stop()

    def update_parts(self):
        """
        loop over all parts
        """
        for entry in self.parts:
            # no funciona si hay una condición de ejecución que es falsa
            run = True
            if entry.get('run_condition'):
                run_condition = entry.get('run_condition')
                run = self.mem.get([run_condition])[0]
                
            if run:
                p = entry['part']
                # obtener entradas de la memoria
                inputs = self.mem.get(entry['inputs'])

                # ejecutar el componente
                if entry.get('thread'):
                    outputs = p.run_threaded(*inputs)
                else:
                    outputs = p.run(*inputs)

                # guardar la salida en la memoria
                if outputs is not None:
                    self.mem.put(entry['outputs'], outputs)

    def stop(self):
        logger.info('Shutting down vehicle and its parts...')
        for entry in self.parts:
            try:
                entry['part'].shutdown()
            except Exception as e:
                logger.debug(e)
